In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from requests import Request
from client import FtxClient

# to reduce amount of repeating code from notebook to another there is a lib of
#often used functions called
import sys
sys.path.insert(0, "C:/workspace/data-science/perpetual/second_investigation_wave")
from data_transformers import *

# making a function for collecting price history for entire year

In [14]:
ENDPOINT = 'https://ftx.com/api'
FTX_API = "SCmQmMDbKXS9H3p7HpUOP7BW7UnbKupWeo9P91If"

def get_ftx_price_story_for_year_per_minute(market_name: str, start_time: int, offset: int=7200, days_in_year: int=365):
    """get price of the token from FTX platform for one year per each minute

    Args:
        market_name (str): name of market for which price it is required to find
        start_time (int): start time in second-based timestamp
        offset (int, optional): offset to correct timezone difference in seconds. Defaults to 7200.
        days_in_year (int, optional): amount of days in year. Defaults to 365.

    Returns:
        _type_: _description_
    """
    list_of_daily_stories = []
    resolution = 60
    start_timestamp = start_time + offset
    seconds_per_day = 86400
    
    for day in range(days_in_year):
        path = ("/markets/" + market_name + "/candles?resolution=" + str(resolution) + "&" +
                "start_time=" + str(start_timestamp + seconds_per_day * day) + "&end_time=" + 
                str(start_timestamp + seconds_per_day * (day + 1)))  
        url = ENDPOINT + path
        res = requests.get(url).json()
        df = pd.DataFrame(res['result'])
        df['date'] = pd.to_datetime(df['startTime'])
        list_of_daily_stories.append(df)
        
    return pd.concat(list_of_daily_stories)

In [15]:
eth_ftx_price_2021_df = get_ftx_price_story_for_year_per_minute("ETH/USD", datetime(2021, 1, 1).timestamp(),
                                                                7200)
eth_ftx_price_2021_df

,startTime,time,open,high,low,close,volume,date
0,2021-01-01T00:00:00+00:00,1.609459e+12,737.28,737.91,736.90,737.80,9917.16625,2021-01-01 00:00:00+00:00
1,2021-01-01T00:01:00+00:00,1.609459e+12,737.80,739.92,737.80,739.53,30086.73403,2021-01-01 00:01:00+00:00
2,2021-01-01T00:02:00+00:00,1.609459e+12,739.53,740.00,738.21,738.75,257.16222,2021-01-01 00:02:00+00:00
3,2021-01-01T00:03:00+00:00,1.609459e+12,738.75,739.10,738.14,738.14,185.36104,2021-01-01 00:03:00+00:00
4,2021-01-01T00:04:00+00:00,1.609459e+12,738.14,738.63,737.62,737.78,178.66611,2021-01-01 00:04:00+00:00
...,...,...,...,...,...,...,...,...
1429,2021-12-31T23:56:00+00:00,1.640995e+12,3677.30,3678.90,3677.20,3677.40,96718.27870,2021-12-31 23:56:00+00:00
1430,2021-12-31T23:57:00+00:00,1.640995e+12,3677.40,3679.30,3677.40,3677.70,35194.77190,2021-12-31 23:57:00+00:00
1431,2021-12-31T23:58:00+00:00,1.640995e+12,3677.70,3678.60,3672.60,3675.50,293073.79940,2021-12-31 23:58:00+00:00
1432,2021-12-31T23:59:00+00:00,1.640995e+12,3675.50,3678.60,3675.20,3676.50,138598.98440,2021-12-31 23:59:00+00:00


In [16]:
eth_ftx_price_2021_df.sort_values(by="time", inplace=True)
eth_ftx_price_2021_df.reset_index(inplace=True)
eth_ftx_price_2021_df.drop(columns=["index", "date"], inplace=True)
eth_ftx_price_2021_df["time"] = pd.to_datetime(eth_ftx_price_2021_df["startTime"]
                                                ).dt.tz_localize(None)
eth_ftx_price_2021_df.drop(columns=["startTime"], inplace=True)
eth_ftx_price_2021_df

,startTime,time,open,high,low,close,volume
0,2021-01-01T00:00:00+00:00,1.609459e+12,737.28,737.91,736.90,737.80,9917.16625
1,2021-01-01T00:01:00+00:00,1.609459e+12,737.80,739.92,737.80,739.53,30086.73403
2,2021-01-01T00:02:00+00:00,1.609459e+12,739.53,740.00,738.21,738.75,257.16222
3,2021-01-01T00:03:00+00:00,1.609459e+12,738.75,739.10,738.14,738.14,185.36104
4,2021-01-01T00:04:00+00:00,1.609459e+12,738.14,738.63,737.62,737.78,178.66611
...,...,...,...,...,...,...,...
525875,2021-12-31T23:56:00+00:00,1.640995e+12,3677.30,3678.90,3677.20,3677.40,96718.27870
525876,2021-12-31T23:57:00+00:00,1.640995e+12,3677.40,3679.30,3677.40,3677.70,35194.77190
525877,2021-12-31T23:58:00+00:00,1.640995e+12,3677.70,3678.60,3672.60,3675.50,293073.79940
525878,2021-12-31T23:59:00+00:00,1.640995e+12,3675.50,3678.60,3675.20,3676.50,138598.98440


In [17]:
btc_ftx_price_2021_df = get_ftx_price_story_for_year_per_minute("BTC/USD", datetime(2021, 1, 1).timestamp(),
                                                                7200)
btc_ftx_price_2021_df

,startTime,time,open,high,low,close,volume,date
0,2021-01-01T00:00:00+00:00,1.609459e+12,28965.0,29014.5,28960.0,29014.5,1.468796e+05,2021-01-01 00:00:00+00:00
1,2021-01-01T00:01:00+00:00,1.609459e+12,29014.5,29070.5,29014.5,29065.5,5.276008e+05,2021-01-01 00:01:00+00:00
2,2021-01-01T00:02:00+00:00,1.609459e+12,29065.5,29066.0,29018.0,29041.5,1.813415e+05,2021-01-01 00:02:00+00:00
3,2021-01-01T00:03:00+00:00,1.609459e+12,29041.5,29050.5,29022.0,29028.0,3.602541e+05,2021-01-01 00:03:00+00:00
4,2021-01-01T00:04:00+00:00,1.609459e+12,29028.0,29040.5,29021.5,29022.0,2.229805e+05,2021-01-01 00:04:00+00:00
...,...,...,...,...,...,...,...,...
1429,2021-12-31T23:56:00+00:00,1.640995e+12,46244.0,46270.0,46238.0,46242.0,1.430735e+05,2021-12-31 23:56:00+00:00
1430,2021-12-31T23:57:00+00:00,1.640995e+12,46242.0,46269.0,46236.0,46237.0,1.526918e+05,2021-12-31 23:57:00+00:00
1431,2021-12-31T23:58:00+00:00,1.640995e+12,46237.0,46237.0,46136.0,46179.0,1.191280e+06,2021-12-31 23:58:00+00:00
1432,2021-12-31T23:59:00+00:00,1.640995e+12,46179.0,46225.0,46177.0,46199.0,2.645854e+05,2021-12-31 23:59:00+00:00


In [20]:
btc_ftx_price_2021_df.sort_values(by="time", inplace=True)
btc_ftx_price_2021_df.reset_index(inplace=True)
btc_ftx_price_2021_df.drop(columns=["index", "date"], inplace=True)
btc_ftx_price_2021_df["time"] = pd.to_datetime(btc_ftx_price_2021_df["startTime"]
                                                ).dt.tz_localize(None)
btc_ftx_price_2021_df.drop(columns=["startTime"], inplace=True)
btc_ftx_price_2021_df

,time,open,high,low,close,volume
0,2021-01-01 00:00:00,28965.0,29014.5,28960.0,29014.5,1.468796e+05
1,2021-01-01 00:01:00,29014.5,29070.5,29014.5,29065.5,5.276008e+05
2,2021-01-01 00:02:00,29065.5,29066.0,29018.0,29041.5,1.813415e+05
3,2021-01-01 00:03:00,29041.5,29050.5,29022.0,29028.0,3.602541e+05
4,2021-01-01 00:04:00,29028.0,29040.5,29021.5,29022.0,2.229805e+05
...,...,...,...,...,...,...
525872,2021-12-31 23:56:00,46244.0,46270.0,46238.0,46242.0,1.430735e+05
525873,2021-12-31 23:57:00,46242.0,46269.0,46236.0,46237.0,1.526918e+05
525874,2021-12-31 23:58:00,46237.0,46237.0,46136.0,46179.0,1.191280e+06
525875,2021-12-31 23:59:00,46179.0,46225.0,46177.0,46199.0,2.645854e+05


In [22]:
perp_ftx_price_2021_df = get_ftx_price_story_for_year_per_minute("PERP/USD", datetime(2021, 3, 1).timestamp(),
                                                                7200)
perp_ftx_price_2021_df

,startTime,time,open,high,low,close,volume,date
0,2021-03-01T00:00:00+00:00,1.614557e+12,4.609,4.624,4.609,4.624,1120.2155,2021-03-01 00:00:00+00:00
1,2021-03-01T00:01:00+00:00,1.614557e+12,4.624,4.655,4.624,4.645,937.8225,2021-03-01 00:01:00+00:00
2,2021-03-01T00:02:00+00:00,1.614557e+12,4.645,4.648,4.635,4.635,321.7042,2021-03-01 00:02:00+00:00
3,2021-03-01T00:03:00+00:00,1.614557e+12,4.635,4.635,4.629,4.629,481.6378,2021-03-01 00:03:00+00:00
4,2021-03-01T00:04:00+00:00,1.614557e+12,4.629,4.629,4.620,4.624,591.0764,2021-03-01 00:04:00+00:00
...,...,...,...,...,...,...,...,...
1436,2022-02-28T23:56:00+00:00,1.646093e+12,4.690,4.694,4.690,4.694,22.0618,2022-02-28 23:56:00+00:00
1437,2022-02-28T23:57:00+00:00,1.646093e+12,4.694,4.707,4.694,4.699,44.6969,2022-02-28 23:57:00+00:00
1438,2022-02-28T23:58:00+00:00,1.646093e+12,4.699,4.712,4.699,4.712,236.5224,2022-02-28 23:58:00+00:00
1439,2022-02-28T23:59:00+00:00,1.646093e+12,4.712,4.712,4.708,4.708,0.0000,2022-02-28 23:59:00+00:00


In [23]:
perp_ftx_price_2021_df.sort_values(by="time", inplace=True)
perp_ftx_price_2021_df.reset_index(inplace=True)
perp_ftx_price_2021_df.drop(columns=["index", "date"], inplace=True)
perp_ftx_price_2021_df["time"] = pd.to_datetime(perp_ftx_price_2021_df["startTime"]
                                                ).dt.tz_localize(None)
perp_ftx_price_2021_df.drop(columns=["startTime"], inplace=True)
perp_ftx_price_2021_df

,time,open,high,low,close,volume
0,2021-03-01 00:00:00,4.609,4.624,4.609,4.624,1120.2155
1,2021-03-01 00:01:00,4.624,4.655,4.624,4.645,937.8225
2,2021-03-01 00:02:00,4.645,4.648,4.635,4.635,321.7042
3,2021-03-01 00:03:00,4.635,4.635,4.629,4.629,481.6378
4,2021-03-01 00:04:00,4.629,4.629,4.620,4.624,591.0764
...,...,...,...,...,...,...
525878,2022-02-28 23:56:00,4.690,4.694,4.690,4.694,22.0618
525879,2022-02-28 23:57:00,4.694,4.707,4.694,4.699,44.6969
525880,2022-02-28 23:58:00,4.699,4.712,4.699,4.712,236.5224
525881,2022-02-28 23:59:00,4.712,4.712,4.708,4.708,0.0000


# save to csv

In [24]:
eth_ftx_price_2021_df.to_csv("eth_ftx_price_2021.csv")

In [25]:
btc_ftx_price_2021_df.to_csv("btc_ftx_price_2021.csv")

In [26]:
perp_ftx_price_2021_df.to_csv("perp_ftx_price_2021.csv")